Implement a CNN on Tomato dataset using batch sizes of 32 and 64 separately. Keep the learning 
rate fixed at 0.0001 and compare results. 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os

IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZES = [32, 64]
EPOCHS = 20
LEARNING_RATE = 0.0001

train_dir = '/home/adithya/Deep_Learning/DL3_Dataset/Tomato/Train'
val_dir = '/home/adithya/Deep_Learning/DL3_Dataset/Tomato/Val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

def build_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def plot_history(histories, batch_sizes):
    plt.figure(figsize=(12, 5))
    for i, history in enumerate(histories):
        plt.subplot(1, 2, 1)
        plt.plot(history.history['val_accuracy'], label=f'Batch {batch_sizes[i]}')
        plt.title('Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['val_loss'], label=f'Batch {batch_sizes[i]}')
        plt.title('Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    if not os.path.exists(train_dir):
        raise FileNotFoundError(f"Training directory '{train_dir}' does not exist. Please check your dataset path.")

    class_folders = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
    if not class_folders:
        raise ValueError(f"No class subfolders found in '{train_dir}'. Please organize your dataset as '{train_dir}/class_name/'.")
    num_classes = len(class_folders)
    print(f'Number of classes: {num_classes}')

    histories = []

    for batch_size in BATCH_SIZES:
        print(f"\nTraining with batch size: {batch_size}")
        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(IMG_HEIGHT, IMG_WIDTH),
            batch_size=batch_size,
            class_mode='categorical'
        )
        val_generator = val_datagen.flow_from_directory(
            val_dir,
            target_size=(IMG_HEIGHT, IMG_WIDTH),
            batch_size=batch_size,
            class_mode='categorical'
        )

        model = build_cnn_model((IMG_HEIGHT, IMG_WIDTH, 3), num_classes)
        model.compile(
            optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        history = model.fit(
            train_generator,
            epochs=EPOCHS,
            validation_data=val_generator,
            verbose=1
        )
        histories.append(history)

    plot_history(histories, BATCH_SIZES)

    for i, batch_size in enumerate(BATCH_SIZES):
        val_acc = histories[i].history['val_accuracy'][-1]
        print(f"Batch Size {batch_size}: Final Validation Accuracy = {val_acc:.4f}")

Number of classes: 6

Training with batch size: 32
Found 11108 images belonging to 6 classes.
Found 2495 images belonging to 6 classes.


/home/adithya/Deep_Learning/tensorflow_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1747944975.732436   14568 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/adithya/Deep_Learning/tensorflow_env/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_sup

Epoch 1/20


I0000 00:00:1747944979.245635   16039 service.cc:152] XLA service 0x7fe500004ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747944979.245703   16039 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-05-22 20:16:19.313583: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747944979.672588   16039 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-22 20:16:20.527375: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.9 = (f32[32,32,126,126]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,128,128]{3,2,1,0} %bitcast.4699, f32[32,3,3,3]{3,2,1,0} %bitcast.4706, f32[32]{0} %bitcast.5244), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$con

348/348 - 61s - 175ms/step - accuracy: 0.3972 - loss: 1.4675 - val_accuracy: 0.4958 - val_loss: 1.2726
Epoch 2/20
348/348 - 42s - 119ms/step - accuracy: 0.6385 - loss: 0.9732 - val_accuracy: 0.6048 - val_loss: 1.0871
Epoch 3/20
348/348 - 40s - 116ms/step - accuracy: 0.7069 - loss: 0.8045 - val_accuracy: 0.7255 - val_loss: 0.7153
Epoch 4/20
348/348 - 40s - 116ms/step - accuracy: 0.7396 - loss: 0.7161 - val_accuracy: 0.6509 - val_loss: 1.0335
Epoch 5/20
348/348 - 42s - 120ms/step - accuracy: 0.7681 - loss: 0.6480 - val_accuracy: 0.6092 - val_loss: 1.2351
Epoch 6/20
348/348 - 44s - 127ms/step - accuracy: 0.7891 - loss: 0.6052 - val_accuracy: 0.5651 - val_loss: 1.3666
Epoch 7/20
348/348 - 58s - 166ms/step - accuracy: 0.8046 - loss: 0.5515 - val_accuracy: 0.6846 - val_loss: 1.0316
Epoch 8/20
348/348 - 64s - 184ms/step - accuracy: 0.8147 - loss: 0.5164 - val_accuracy: 0.7042 - val_loss: 0.9786
Epoch 9/20
348/348 - 69s - 200ms/step - accuracy: 0.8275 - loss: 0.4938 - val_accuracy: 0.7687 - va